# `Практикум по программированию на языке Python`
<br>

## `Занятие 13: Серверная Web-разработка`
<br><br>

### `Роман Ищенко (roman.ischenko@gmail.com)`

#### `Москва, 2022`

In [1]:
import warnings
warnings.filterwarnings('ignore')

### `Python web-server libs`

#### `Flask`
Flask — микрофреймворк для создания вебсайтов на языке Python.

In [ ]:
# Простейший сервер на flask
from flask import Flask

app = Flask(__name__)

@app.route('/', methods=['GET'])
def hello_world():
   return 'Hello, World!'
   
@app.route('/post')
def hello_path():
   return 'Hello, Path!'
   
if __name__ == '__main__':
   app.run()

In [8]:
# Можно открыть в браузере или написать простейший клиент на Python
import requests
r = requests.get('http://127.0.0.1:5000/post')
print(r.status_code)
print(r.headers)
print(r.content)

200
{'Server': 'Werkzeug/2.1.2 Python/3.8.12', 'Date': 'Wed, 09 Nov 2022 17:54:10 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '12', 'Connection': 'close'}
b'Hello, Path!'


По умолчанию `route` отвечает только на `GET` запросы.<br>
Если нужно, можно явно добавить HTTP-методы, которые будут обрабатываться


In [ ]:
from flask import Flask, request

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def hello_world():
    print(request.method)
    return {'data': 'Hello, World!'}
   
if __name__ == '__main__':
   app.run()

В пути можно использовать переменные

Синтаксис: `<converter:variable_name>`

Доступные converters:
- string
- int
- float
- path
- uuid


In [ ]:
@app.route('/hello/<string:name>')
def hello_name(name):
    return f'Hello {name}!'

In [11]:
import requests
r = requests.get('http://127.0.0.1:5000/hello/John')
print(r.content)

b'Hello, John!'


Flask используется для разработки и отладки.

Для промышленной эксплуатации необходимо использование WSGI (Web Server Gateway Interface) сервера:
- WSGI-сервера были разработаны чтобы обрабатывать множество запросов одновременно. А фреймворки (в том числе flask) не предназначены для обработки тысяч запросов и не дают решения того, как наилучшим образом маршрутизировать запросы с веб-сервера.
- с WSGI  не нужно беспокоиться о том, как ваша конкретная инфраструктура использует стандарт WSGI.
- WSGI дает Вам гибкость в изменении компонентов веб-стека без изменения приложения, которое работает с WSGI.

Если не планируется большой нагрузки, для `flask` это может быть `waitress`.

Установка: `pipenv install waitress`

Использование:

In [ ]:
from waitress import serve
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello_world():
   return 'Hello, World!'
   
if __name__ == '__main__':
    # Вместо запуска flask запускаем waitress.serve
    # app.run(threaded=False, processes=2)
    #  serve(app, host='0.0.0.0', port='5000', threads=2, connection_limit=4)
    serve(app, host='0.0.0.0', port='5000')


Либо запускаем из командной строки: `waitress-serve --port 5000 '<имя модуля>:<перемнная приложения>'`

Если наш файл называется `server.py`, то наш пример можно запустить командой: `waitress-serve --port 5000 'server:app'`

#### `FastAPI`

FastAPI — фреймворк для создания лаконичных и довольно быстрых HTTP API-серверов

In [1]:
from typing import Optional
from fastapi import FastAPI

app = FastAPI()


@app.get("/")
def hello_world(q: Optional[str] = None):
    return {'data': f'Hello, World! Param: {q}'}


@app.post("/items/{item_id}")
@app.get("/items/{item_id}")
def read_item(item_id: int, a: int, q: Optional[str] = None):
    return {"item_id": item_id, "q": q}


In [ ]:
from typing import Optional

from fastapi import FastAPI
from pydantic import BaseModel


class Item(BaseModel):
    name: str
    description: Optional[str] = None


app = FastAPI()


@app.post("/")
async def create_item(item: Item):
    return item

In [3]:
import requests
r = requests.post('http://127.0.0.1:8000/', json={'name1': 'John'})
print(r.status_code, r.content)

200 b'{"name":"John","description":null}'


FastAPI требует уже ASGI (Asynchronous Standard Gateway Interface) сервера, например, uvicorn

Запуск приложения: `uvicorn '<имя модуля>:<перемнная приложения>'`

Все запросы к веб-сервису выполняются последовательно. Можно использовать асинхронность и многопоточность, но мы знаем, что она сработает не во всех случаях.

Эту проблему решают масштабированием через внешние WSGI-серверы. Для Python их существует некоторое количество: Bjoern, uWSGI, mod_wsgi, Meinheld, CherryPy, Gunicorn.

Gunicorn — это WSGI-сервер, созданный для использования в UNIX-системах. Название — сокращенная и комбинированная версия слов «Green Unicorn». На самом сайте проекта есть зеленый единорог. Gunicorn был перенесен из проекта «Unicorn» из языка Ruby. Он относительно быстрый, не требует много ресурсов, легко запускается и работает с широким спектром веб-фреймворков.

<img src="https://cdn-images-1.medium.com/max/1200/1*nFxyDwJ2DEH1G5PMKPMj1g.png"/>

Запуск для нашего примера для Flask: `gunicorn --bind 0.0.0.0:5000 --workers 4 'server:app'`

Запуск для нашего примера для FastAPI:`gunicorn --access-logfile - --bind 0.0.0.0:8000 --workers 2 -k uvicorn.workers.UvicornWorker server:app`

## `Спасибо за внимание!`